In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from model import ResNet20
from Train import model_training

In [2]:
# Hyper parameters
epoch_losses = []
train_loss_list = []
epochs = 50
lr = 0.01
batchsize = 64

# Model device assignment
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ResNet20().to(device)

In [3]:
# NDefine transformation
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

# Load the training dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

# Load the test dataset
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

# Define the class names for reference
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [4]:
# Testing model on Cifar10
testing_module = model_training(model, lr, trainloader, epochs)
train_loss = testing_module.forward(device)

Start Training for 50 epochs
training cycle: 0


RuntimeError: The size of tensor a (7) must match the size of tensor b (16) at non-singleton dimension 3

In [ ]:
# Visualization of training results 
plt.plot(train_loss, epochs, linewidth=2, c='r')
plt.xlabel("Epochs")
plt.ylabel("Training Loss")
plt.title("Training Loss of ResNet20 Model")
plt.grid(True)

plt.tight_layout()
plt.show()